In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/local_Documents/TheoryAndPracticeOfCausalInference`


In [2]:
Pkg.add(["CSV", "DataFrames", "HTTP", "Statistics"])
using CSV, DataFrames, HTTP, Statistics

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Project.toml`
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Manifest.toml`


In [3]:
data02 = CSV.read(HTTP.get("https://raw.githubusercontent.com/mtakahashi123/causality/main/data02.csv").body, DataFrame; missingstring = "NA")

,x1,y3,t1,y0,y1,y0t,y1t
,Int64,Int64,Int64,Int64?,Int64?,Int64,Int64
1,74,76,1,missing,76,68,76
2,82,75,0,75,missing,75,84
3,72,75,1,missing,75,65,75
4,96,84,0,84,missing,84,97
5,83,75,0,75,missing,75,84
6,72,74,1,missing,74,65,74
7,85,76,0,76,missing,76,87
8,87,77,0,77,missing,77,89
9,86,77,0,77,missing,77,87


In [4]:
describe(data02) # 記述統計量の算出

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Int64,Float64,Int64,Int64,Type
1,x1,81.95,58,83.5,96,0,Int64
2,y3,76.6,61,76.5,87,0,Int64
3,t1,0.3,0,0.0,1,0,Int64
4,y0,77.7857,72,77.0,87,6,"Union{Missing, Int64}"
5,y1,73.8333,61,75.5,80,14,"Union{Missing, Int64}"
6,y0t,73.8,52,75.0,87,0,Int64
7,y1t,83.85,61,84.5,97,0,Int64


- x1~ Normal(80, 10)
- t1 -> 処置割り付けの二値変数
- y0t ~ Normal(80, 9)
- y1t ~ Normal(90, 9)
- y0 x1が80以上のときに観測される変数（期末試験0, Y|T=0）
- y1 x1が80未満のときに観測される変数（期末試験1, Y|T=1)
- y3 y0とy1の観測部分をくっつけた変数

## 処置効果1：個体因果効果
Individual Causal Effect, ICE, もしくはIndividual Treatment Effect, ITE

ICEは単純に，個体ごとの「潜在的結果1-潜在的結果0」と定義される。

In [5]:
# せっかくなのでattach関数を実装する。
function attach(x::DataFrame)
    for i in propertynames(x)
        @eval $(i) = $(x).$(i)
    end
end

attach (generic function with 1 method)

In [6]:
attach(data02)

個体処置効果, ICE

定義はできても，一部に観測できない量があるため，推定できない。

In [7]:
y1t - y0t

20-element Vector{Int64}:
  8
  9
 10
 13
  9
  9
 11
 12
 10
 10
  9
 10
 10
  9
 12
 12
 10
  9
 10
  9

## 処置効果2：平均処置効果

Average Treatment Effect

期待値の加法性から和の期待値は期待値の和に変形できる。

$$
\tau = E[Y_i(1) - Y_i(0)] = E[Y_i(1)] - E[Y_i(0)]
$$

実際には同一個体の$Y_i(1)$と$Y_i(0)$は同時に観測されないので，直接ATEを計算することはできない。

In [8]:
mean(y1t) - mean(y0t)

10.049999999999997

In [9]:
mean(y3) - mean(x1)　# 実際に観測できるデータから計算した事前事後比較

-5.3500000000000085

In [10]:
# ナイーブにペアワイズ除去した場合の，期末試験得点の差
m1 = mean(skipmissing(y1)) 
m0 = mean(skipmissing(y0))
m1 - m0

-3.952380952380963

## 処置効果3：処置群の平均処置効果

Averate Treatment effect on the Treated, ATE

実際に処置を受けた群における効果（政策評価や心理実験などでの関心）

$$
\tau_{ATT} = E[Y_i(1) - Y_i(0)|T_i = 1] = E[Y_i(1)|T_i = 1] - E[Y_i(0)|T_i = 1]
$$

ATEとATTのどちらを使うかは研究テーマによる。手法によってもどちらが推定可能かが変わってくる。

In [11]:
mt1 = mean(y1t[@. t1 == 1])
mt0 = mean(y0t[@. t1 == 1])
mt1 - mt0

9.333333333333329

## 交絡因子

ナイーブな推定量$(y3 - x1)$における推論が「補習ををすると得点が下がる」と結論付けられてしまう。

= 何らかのcofoundingが働いている。

= そもそも補習を受ける人たちは数学が苦手な傾向にあるはずなので，補習授業の効果があったとしても，それを覆い隠すほどに初期位置の数学力が低い可能性がある。

## DAG

## 無作為割付けによる分析の例

In [12]:
Pkg.add(["Random", "Distributions"])
using Random, Distributions

   Resolving package versions...
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Project.toml`
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Manifest.toml`


In [13]:
Random.seed!(1) # 便宜上おなじシードを用いるが，当然Rの結果とは一致しない（乱数生成器も違うし）

TaskLocalRNG()

In [14]:
r0 = rand(Uniform(0, 1), 20)
r1 = @. round(r0; digits = 0)
# r1 = [0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,] # 検算用
y2 = similar(y1t)
y2[@. r1 == 1] = copy(y1t[@. r1 == 1])
y2[@. r1 == 0] = copy(y0t[@. r1 == 0])

10-element Vector{Int64}:
 68
 75
 65
 84
 76
 87
 75
 67
 52
 80

In [15]:
r1'

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  1.0  …  0.0  1.0  1.0  1.0  1.0  0.0  1.0

In [16]:
mr1 = mean(y2[@. r1 == 1])

84.9

In [17]:
mr0 = mean(y2[@. r1 == 0])

72.9

In [18]:
mr1 - mr0 # サンプルサイズが小さすぎるので，結果はかなり不安定。

12.0

## 2標本$t$検定

無作為割付けを実施したデータで2群間に差についてのt検定を行う。Rコードでは`var.equal = FALSE`で，等分散性の仮定を置かないようにしているので，`HypothesisTests.jl`の`UnequalVarianceTTest()`を使う。

In [19]:
Pkg.add("HypothesisTests")
using HypothesisTests

   Resolving package versions...
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Project.toml`
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Manifest.toml`


In [20]:
UnequalVarianceTTest(y2[@. r1 == 1], y2[@. r1 == 0]) # 繰り返すが，サンプルサイズが小さいので同じ結果にはならない。

Two sample t-test (unequal variance)
------------------------------------
Population details:
    parameter of interest:   Mean difference
    value under h_0:         0
    point estimate:          12.0
    95% confidence interval: (4.115, 19.89)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           0.0057

Details:
    number of observations:   [10,10]
    t-statistic:              3.272997779589968
    degrees of freedom:       13.601255666585667
    empirical standard error: 3.666363623840688
